In [1]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import uniform
import numpy as np
import pickle

In [3]:

with open("X_train_scaled.pkl", "rb") as f:
    X_train_scaled = pickle.load(f)

X_train_scaled

array([[ 1.36733216,  1.42484899,  1.39771176, ...,  1.23821508,
         1.03256375, -0.61393228],
       [-1.64262117, -1.38821735, -1.69983868, ..., -1.65940326,
        -1.34028771, -0.61393228],
       [-0.12491496,  0.23157057,  0.27974756, ...,  0.08400417,
         0.76891358,  1.62884414],
       ...,
       [ 0.87057643,  0.98936095,  0.88373528, ..., -0.66004487,
         0.76891358, -0.61393228],
       [-0.43303746, -0.62438274, -0.1681792 , ...,  0.10019118,
        -0.6811623 , -0.61393228],
       [-0.0053817 , -0.19750606, -0.20137148, ...,  0.31796349,
         1.16438883, -0.61393228]])

In [5]:
with open("X_test_scaled.pkl", "rb") as f:
    X_test_scaled = pickle.load(f)
    
X_test_scaled

array([[-0.60365708,  0.1062256 , -0.48741598, ..., -0.57388629,
        -0.6811623 , -0.61393228],
       [ 1.3989636 ,  1.34885406,  1.28012826, ...,  0.84255533,
         0.10978818, -0.61393228],
       [-0.24874829, -0.62438274, -0.48189014, ..., -0.78401147,
        -0.81298739, -0.61393228],
       ...,
       [ 0.92084444,  0.968366  ,  0.99940568, ...,  1.17614635,
        -0.6811623 , -0.61393228],
       [ 1.22718252,  1.42484899,  1.40183995, ...,  1.45277269,
        -1.20846263, -0.61393228],
       [ 0.40642961, -0.46174402,  0.00713792, ..., -0.61205156,
        -1.20846263,  1.62884414]])

In [7]:
with open("y_train.pkl", "rb") as f:
    y_train = pickle.load(f)
    
y_train

135075    3115.0
48411       56.0
71197      637.0
129903    2298.0
129682     338.0
           ...  
114927     316.0
124946    1782.0
108064    1754.0
137499     577.0
127112     697.0
Name: Actual_trip_time, Length: 110763, dtype: float64

In [15]:
with open("y_test.pkl", "rb") as f:
    y_test = pickle.load(f)
y_test

18349      348.0
53820     2795.0
7120       436.0
56892     3045.0
124410    1466.0
           ...  
137146      67.0
96916       73.0
110154    2175.0
113117    2831.0
69526     1103.0
Name: Actual_trip_time, Length: 27691, dtype: float64

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import uniform
import numpy as np

In [16]:
# Define parameter distribution

param_dist = {'alpha': uniform(0.01,100)}

# Initialize model
lasso = Lasso(max_iter=10000)

# Randomized search
random_search = RandomizedSearchCV(lasso, param_distributions=param_dist,
                                   n_iter=20, cv=5, scoring='neg_mean_squared_error',
                                   random_state=42)
random_search.fit(X_train_scaled, y_train)

# Best model
best_lasso_random = random_search.best_estimator_

# Evaluate
y_pred_random = best_lasso_random.predict(X_test_scaled)
print("RandomizedSearchCV R2 Score:", r2_score(y_test, y_pred_random))
print("RandomizedSearchCV RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_random)))
print("Best alpha:", random_search.best_params_['alpha'])

RandomizedSearchCV R2 Score: 0.8941122710044593
RandomizedSearchCV RMSE: 351.5230770360603
Best alpha: 2.0684494295802445


In [20]:
from sklearn.ensemble import RandomForestRegressor
# Define the model
rf = RandomForestRegressor(random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],            # Number of trees
    'max_depth': [None, 10, 20, 30],            # Depth of tree
    'min_samples_split': [2, 5, 10],            # Min samples to split an internal node
    'min_samples_leaf': [1, 2, 4],              # Min samples at a leaf node
    'max_features': ['sqrt', 'log2',1]          # Features to consider at each split
}

# Grid Search
random_search = RandomizedSearchCV(estimator=rf,
                                   param_distributions=param_grid,
                                   n_iter=20,   # Number of parameter settings that are sampled.
                                   scoring='neg_mean_squared_error',
                                   cv=5,
                                   n_jobs=-1, # Number of jobs to run in parallel( -1 means using all processors)
                                   random_state=42,
                                   verbose=2) # Controls the verbosity: the higher, the more messages.

# Fit on training data (use scaled X_train if scaling was done)
random_search.fit(X_train_scaled, y_train)

# Best model
best_rf = random_search.best_estimator_

# Predictions
y_pred = best_rf.predict(X_test_scaled)

# Evaluation
print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Best Hyperparameters:\n", random_search.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
R² Score: 0.9999764499319337
RMSE: 5.242363214531646


NameError: name 'grid' is not defined

In [22]:
print("Best Hyperparameters:\n", random_search.best_params_)

Best Hyperparameters:
 {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}


**(R-squared) measures how well our model's predictions match the actual values.**

It ranges from:

1.0 → Perfect prediction.

0.0 → Model predicts no better than the mean.

< 0.0 → Model is worse than just predicting the mean.

**Root Mean Squared Error (RMSE)** measures the average magnitude of the prediction error — in the same units as your target variable.

1. It's always non-negative.
2. Lower RMSE = Better fit.
3. If our target variable has a wide range, RMSE of 5.24 might be very small, suggesting excellent predictions.

### Let's check the scale of the Target variable
If our y_test values range from 0 to 1000, RMSE = 5.24 is excellent.

But if the target values are between 0 and 10, it's relatively large.

### Comparison with Standard Deviation of y_test:
If std_dev >> RMSE, your model is performing well.

In [24]:
np.std(y_test)

1080.266787864038

**RMSE is very small compared to std deviation:**

Our target variable naturally varies by ±1080 units, but our model's average error is only ~5.24 units.

That means your model is **capturing almost all of the natural variance in the data.**

### Let’s check for overfitting!

How to Detect Overfitting?

We compare performance on training vs. test data using metrics like:

R² Score

RMSE

Cross-Validation Scores

In [34]:
# 1. Check R² and RMSE on Training Data
y_train_pred = best_rf.predict(X_train_scaled)

print("Train R² Score:", r2_score(y_train, y_train_pred))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))

Train R² Score: 0.9999947593825038
Train RMSE: 2.4671257447789827


In [45]:
# 2 Compare With Test Metrics

**If training R² is also ~0.9999 and RMSE is similar (i.e., no large gap), no overfitting is likely**

So below are the finding for calculating R² and RMSE on Test Data(original way) and on Train Data.


Test R² Score: 0.9999764499319337 | Test RMSE: 5.242363214531646

Train R² Score: 0.9999947593825038 Train RMSE: 2.4671257447789827



In [49]:
# 3 Run K-Fold Cross-Validation

from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(best_rf, X_train_scaled, y_train, cv=5, scoring='r2')
print("Cross-Validated R² Scores:", cv_scores)
print("Mean CV R²:", cv_scores.mean())

Cross-Validated R² Scores: [0.99996685 0.99996446 0.99996011 0.99996202 0.99996193]
Mean CV R²: 0.999963075115385


1. Extremely High R² Across All Folds:-
   
**All the values are consistently around 0.99996, which is very stable. No large drop in any fold.**

2. Low Variance Between Folds
   
**The difference between the best and worst R² is very small (~0.000006), meaning:** 

Our model generalizes well across different splits of the data and It's not overly sensitive to specific subsets of our training data



## Is our Model overfitting?

Mean R² (CV) ≈ Test R² ≈ Train R²

1. Train R²: ~0.999994
2. Test R²: ~0.999976
3. Mean CV R²: ~0.999963
   
This consistency across training, validation, and testing is a strong sign of a **robust model**





Our model is **not overfitting** in any harmful way.

It may appear to be “too good,” but the cross-validation and test metrics both support its strong generalization.

In [11]:
pip install lightgbm



   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.8 MB/s eta 0:00:00


In [17]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Step 1: Define the model
lgbm = LGBMRegressor(random_state=42)

# Step 2: Hyperparameter search space
param_dist = {
    'n_estimators': [100, 200, 500, 1000],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'max_depth': [-1, 5, 10, 15],
    'num_leaves': [31, 50, 100, 150],
    'min_child_samples': [5, 10, 20, 50],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Step 3: RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_dist,
    n_iter=30,
    scoring='r2',
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

# Step 4: Fit on training data
random_search.fit(X_train_scaled, y_train)

# Step 5: Evaluate
best_lgbm = random_search.best_estimator_

# Predictions
y_pred = best_lgbm.predict(X_test_scaled)

# Metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Best Hyperparameters:", random_search.best_params_)
print("R² Score on Test:", r2)
print("RMSE on Test:", rmse)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3361
[LightGBM] [Info] Number of data points in the train set: 110763, number of used features: 18
[LightGBM] [Info] Start training from score 1114.145143


F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Best Hyperparameters: {'subsample': 1.0, 'num_leaves': 150, 'n_estimators': 1000, 'min_child_samples': 20, 'max_depth': 15, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
R² Score on Test: 0.9999992991250947
RMSE on Test: 0.9043806869540412


In [19]:
import pickle

# Save the trained model
with open("best_lgbm_model.pkl", "wb") as f:
    pickle.dump(best_lgbm, f)

### Let’s check for if LightGbm model is overfitting!

In [27]:
# 1. Check R² and RMSE on Training Data
y_train_pred = best_lgbm.predict(X_train_scaled)

print("Train R² Score:", r2_score(y_train, y_train_pred))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))

F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Train R² Score: 0.9999998776652881
Train RMSE: 0.3769423256876934


In [30]:
# 2. Use a Validation Set or Cross-Validation - Instead of only train/test, split into train/valid/test or use K-Fold CV:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(best_lgbm, X_train_scaled, y_train, cv=5, scoring='r2')
print("Cross-Validated R² Scores:", cv_scores)
print("Mean CV R²:", cv_scores.mean())

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018657 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3359
[LightGBM] [Info] Number of data points in the train set: 88610, number of used features: 18
[LightGBM] [Info] Start training from score 1116.124376
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3361
[LightGBM] [Info] Number of data points in the train set: 88610, number of used features: 18
[LightGBM] [Info] Start training from score 1113.480155
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017741 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3357
[LightGBM] [Info] Number of data points in the train set: 88610, number of used features: 18
[LightGBM] [Info] Start training from score 1113.058086
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016874 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3357
[LightGBM] [Info] Number of data points in the train set: 88611, number of used features: 18
[LightGBM] [Info] Start training from score 1113.827031
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014786 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3357
[LightGBM] [Info] Number of data points in the train set: 88611, number of used features: 18
[LightGBM] [Info] Start training from score 1114.236071
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

F:\New folder\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Cross-Validated R² Scores: [0.99999888 0.99999875 0.99999848 0.99999865 0.99999769]
Mean CV R²: 0.9999984912189991
